In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/IA/PracticaCalificada2-CC421/train.csv', chunksize=42000000)

In [ ]:
data.get_chunk()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
41999995,2012-01-27 14:35:43.0000006,4.5,2012-01-27 14:35:43 UTC,-73.955099,40.773177,-73.959940,40.766715,1
41999996,2014-07-17 13:29:00.00000079,3.5,2014-07-17 13:29:00 UTC,-73.980840,40.765832,-73.983065,40.766787,2
41999997,2014-10-05 12:53:00.00000069,5.0,2014-10-05 12:53:00 UTC,-73.959443,40.780072,-73.970320,40.786592,3
41999998,2010-11-28 18:47:42.0000006,4.1,2010-11-28 18:47:42 UTC,-73.955497,40.764029,-73.950687,40.771129,1
